In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, udf
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType

# Spark Session
spark = SparkSession.builder \
    .appName("RedditKafkaConsumer") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [2]:
df = spark.read.parquet("/home/jovyan/stream-output/parquet/*")  # Using wildcard to skip metadata folder
df.show(10)

+-------+--------------------+------------------+-------------+-----+-----------------+--------------------+--------------------+---------------+
|     id|                body|            author|  created_utc|score|        subreddit|           permalink|     sentiment_score|sentiment_label|
+-------+--------------------+------------------+-------------+-----+-----------------+--------------------+--------------------+---------------+
|mzc4d1m|        Donald Trump| Ok-Importance8753|1.750689657E9|    1|        AskReddit|/r/AskReddit/comm...|                 0.0|        neutral|
|mzc4ed3|It's important to...|         Shapen361|1.750689667E9|    1|           stocks|/r/stocks/comment...|0.016888888888888873|        neutral|
|mzc4lqq|Of course they sh...|      changelingcd|1.750689727E9|    1|           canada|/r/canada/comment...|-0.19270833333333331|       negative|
|mzc4meg|> Shaping the Con...|        HaLoGuY007|1.750689732E9|    1|    foreignpolicy|/r/foreignpolicy/...| 0.0588695286195

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    from_unixtime, to_date, hour, dayofweek, length, col
)

# Load data
df = spark.read.parquet("/home/jovyan/stream-output/parquet/*")

# Basic cleaning
df_clean = df.dropna(subset=["body", "author", "created_utc", "sentiment_label"])

# Feature engineering
df_transformed = df_clean \
    .withColumn("created_datetime", from_unixtime("created_utc")) \
    .withColumn("date", to_date(col("created_datetime"))) \
    .withColumn("hour", hour(col("created_datetime"))) \
    .withColumn("day_of_week", dayofweek(col("created_datetime"))) \
    .withColumn("comment_length", length(col("body"))) \
    .withColumn("sentiment_label", col("sentiment_label").cast("string")) \
    .withColumn("score", col("score").cast("int"))

# Optionally cache if doing exploration
df_transformed.cache()

# Show preview
df_transformed.select("id", "subreddit", "sentiment_label", "score", "comment_length").show(10)

# Save as new Parquet or database table
df_transformed.write.mode("overwrite").parquet("/home/jovyan/cleaned/")

+-------+-----------------+---------------+-----+--------------+
|     id|        subreddit|sentiment_label|score|comment_length|
+-------+-----------------+---------------+-----+--------------+
|mzc4d1m|        AskReddit|        neutral|    1|            12|
|mzc4ed3|           stocks|        neutral|    1|           869|
|mzc4lqq|           canada|       negative|    1|           913|
|mzc4meg|    foreignpolicy|        neutral|    1|          6351|
|mzc4ozi|   wallstreetbets|        neutral|    1|           386|
|mzc4prc|   NoShitSherlock|        neutral|    1|           132|
|mzc4rmk|InternationalNews|       positive|    1|           250|
|mzc4vlr|           canada|       positive|    1|            96|
|mzc4wwx| moderatepolitics|       positive|    1|           363|
|mzc4y8g|  fivethirtyeight|       positive|    1|           439|
+-------+-----------------+---------------+-----+--------------+
only showing top 10 rows



In [4]:
import pandas as pd

# Convert to Pandas
df_pandas = df_transformed.toPandas()

# Use clickhouse-connect (pip install clickhouse-connect)
import clickhouse_connect

client = clickhouse_connect.get_client(host='clickhouse-server', port=8123, username='default', password='1234')

client.command("""
CREATE TABLE IF NOT EXISTS reddit_cleaned (
    id String,
    body String,
    author String,
    created_utc UInt32,
    score Int32,
    subreddit String,
    permalink String,
    sentiment_score Float32,
    sentiment_label String,
    created_datetime DateTime,
    date Date,
    hour UInt8,
    day_of_week UInt8,
    comment_length UInt16
) ENGINE = MergeTree()
ORDER BY (date, subreddit)
""")

# Convert the 'created_datetime' column to datetime type if it's not already
df_pandas['created_datetime'] = pd.to_datetime(df_pandas['created_datetime'])

client.insert_df("reddit_cleaned", df_pandas)


In [5]:
df = client.query_df('SELECT * FROM reddit_cleaned LIMIT 10')
print(df.head())

        id                                               body  \
0  mzc6cbx  I don‘t know how smart it is to ask Trump for ...   
1  mzc6e7t  Trump doesn't want a full scale war. There's a...   
2  mzc8mno  So what they are just pull everything out of t...   
3  mzcjyy7  They aren't silent but MSM is not amplifying a...   
4  mzcka7d    Wow, he sound so similar to Trump when speaking   

                 author  created_utc  score         subreddit  \
0          ReadyLab5110   1750690236      1  2westerneurope4u   
1        HereIGoAgain99   1750690252      1  2westerneurope4u   
2              Sciomnia   1750690901      1  2westerneurope4u   
3  Equivalent_Nerve_870   1750694125      1             50501   
4         Low-Slide4516   1750694211      1               70s   

                                           permalink  sentiment_score  \
0  /r/2westerneurope4u/comments/1liierd/germany_m...         0.357143   
1  /r/2westerneurope4u/comments/1ligwhe/its_time/...         0.175000   